In [18]:
def nfdump_filter(protocols = [], src_ips = [], src_ports = []):
    """
    Creates an nfdump filter based on a set of characteristics
    :param src_ips:
    :return:
    """
    # Array containing all the individual filters
    output_filter = []

    # Filter by protocol
    if len(protocols) > 0:
        protocol_filter = ["proto " + protocol for protocol in protocols]
        output_filter.append(" or ".join(protocol_filter))

    # Filter by source IP
    if len(src_ips) > 0:
        src_ip_filter = ["src ip " + ip for ip in src_ips]
        output_filter.append(" or ".join(src_ip_filter))

    # Filter by port number
    if len(src_ports) > 0:
        src_port_filter = ["src port " + str(port) for port in src_ports]
        output_filter.append(" or ".join(src_port_filter))

    return "(" + ") and (".join(output_filter) + ")"

In [19]:
from generating_ips import *

In [20]:
ips = generating_ips(200000)

In [21]:
f = open("../input_file_for_test/prod-flow-anon-008.nfdump.filter", "w")
f.write(nfdump_filter(src_ports=[x for x in range(0,65335) if x % 2 == 0], src_ips=ips))
f.close()

In [ ]:
!time nfdump -r ../input_file_for_test/prod-flow-anon-008.nfdump -f ../input_file_for_test/prod-flow-anon-008.nfdump.filter -w ../input_file_for_test/prod-flow-anon-008.nfdump.output

The function above seems to be hitting the limitation of being single threaded, and writing the output after every filter. The execution time with 100 000 IP addresses and 65335 port numbers takes approximately 16 minutes and never exceeds 18% CPU usage, which corresponds with two cores on an eight-core system.

In [8]:
[x for x in range(0, 10) if x % 2 == 0]

[0, 2, 4, 6, 8]